In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

In [3]:
h = 0.1
wavenumber = 0.2

In [4]:
unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_unit_sphere = unit_sphere.vertices
space_unit_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

radius = 4
cube1 = bempp.api.shapes.sphere(r = radius, origin = [6,0,0], h = h)
space_cube1 = bempp.api.function_space(cube1,'P',1)

In [5]:
eta = wavenumber

identity1 = bempp.api.operators.boundary.sparse.identity(space_cube1, space_cube1, space_cube1)
slp1 = bempp.api.operators.boundary.helmholtz.single_layer(space_cube1, space_cube1, space_cube1, wavenumber)
dlp1 = bempp.api.operators.boundary.helmholtz.double_layer(space_cube1, space_cube1, space_cube1, wavenumber)
slp_far1 = helmholtz_farfield.single_layer(space_cube1, vert_unit_sphere, wavenumber)
dlp_far1 = helmholtz_farfield.double_layer(space_cube1, vert_unit_sphere, wavenumber)

lhs1 = 0.5 * identity1 + dlp1 - 1j* eta *slp1

In [6]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*wavenumber*r))*scipy.special.jv(q+0.5, wavenumber*r)


def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [7]:
d = [1,0,0]
def coeff_sph_expansion(p,q):
    """Analytical formula of the coefficients of the spherical expansions of plane waves"""
    coeff = 4*np.pi*(1j)**q*normalized_spherical_harmonics(-p,q,d)/(-1)**(abs(p))
    return coeff

In [9]:
deg = 10

sph_harm_coeff = []
for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_unit_sphere, fun = sph_harm)
        sph_harm_coeff.append(np.conj(sph_harm_grid.coefficients)/((-1j)**(q+1)))

In [10]:
mass_mat = bempp.api.operators.boundary.sparse.identity(space_unit_sphere, space_unit_sphere, space_unit_sphere).weak_form().A

In [11]:
slp_far_field_coeff1 = []
for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def dirichlet_fun(x,n,domain_index,result):
                with objmode():
                    result[0] = - regular_spherical_wavefunctions(p,q,x)\

        rhs_fun1 = bempp.api.GridFunction(space_cube1, fun = dirichlet_fun)
        field1, info1 = bempp.api.linalg.gmres(lhs1, rhs_fun1, tol=1E-8)
        slp_far_field_coeff1.append(dlp_far1.evaluate(field1) - 1j * eta * slp_far1.evaluate(field1))

In [12]:
T_matrix1 = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
for i in range((deg+1)**2):
    for j in range((deg+1)**2):
        T_matrix1[i,j] = (slp_far_field_coeff1[j] @ mass_mat @ sph_harm_coeff[i])*wavenumber

In [13]:
coeff_inc = []
for q in range(deg+1):
    for p in range(-q, q+1):
        coeff_inc.append(coeff_sph_expansion(p,q))

coeff_sca = T_matrix1 @ coeff_inc

In [14]:
def sca_far_field(x):
    fun = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            fun += coeff_sca[i]*(1/wavenumber)*(-1j)**(q+1) * normalized_spherical_harmonics(p, q, x)
            i +=1
    return fun

In [15]:
sca_far_field(vert_unit_sphere[:,1])

(-3.2693396501161143-1.3210565244735992j)

In [16]:
@bempp.api.complex_callable
def exact_data(x, n, domain_index, result):
    result[0] = - np.exp(1j * wavenumber * x[0])

grid_fun = bempp.api.GridFunction(space_cube1, fun=exact_data)
fun_exact, info_exact = bempp.api.linalg.gmres(lhs1, grid_fun, tol=1E-5)
res_exact = dlp_far1.evaluate(fun_exact) - 1j * eta * slp_far1.evaluate(fun_exact)

In [17]:
res_exact[:,1]

array([-3.27488556-1.32486397j])

In [18]:
far_list_test = []
for i in range(vert_unit_sphere.shape[1]):
    far_list_test.append(sca_far_field(vert_unit_sphere[:,i]))


err = np.sqrt((far_list_test - res_exact[0,:]) @ mass_mat @ np.conj(far_list_test - res_exact[0,:]))/np.sqrt(res_exact[0,:]@ mass_mat @ np.conj(res_exact[0,:]))
print(err)

(0.0023760000350179474+1.6208772232362636e-20j)


In [19]:
T_matrix1[1,1]

(-0.10037111579123643-0.15914155820959186j)

In [20]:
T_matrix1[3,3]

(-0.10037143059378693-0.1591427517335943j)

In [22]:
S_matrix = np.identity(T_matrix1.shape[1]) + 2*T_matrix1

In [25]:
np.linalg.norm(S_matrix @ np.matrix(S_matrix).getH() - np.identity(T_matrix1.shape[1]))

0.005387599673725152